In [ ]:
# %% Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from sklearn.decomposition import PCA
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from sklearn.metrics import mean_squared_error

In [ ]:
# %% Define helper functions
def load_color_image(image_path):
    """
    Load an image in color without resizing.
    """
    img = Image.open(image_path)  # keep original colors and size
    return np.array(img)

In [ ]:
# %% Define helper function for Kernel PCA reconstruction
def reconstruct_image_kernel_pca_color(image, n_components=50, kernel="rbf", gamma=0.01):
    """
    Apply Kernel PCA independently to each color channel of the image using the specified kernel.
    Reconstruction requires fit_inverse_transform=True.
    """
    from sklearn.decomposition import KernelPCA

    # Normalize image to [0,1]
    X = image.astype(np.float32) / 255.0
    reconstructed_channels = []

    # Process each channel (assuming the image is in RGB format)
    for channel in range(X.shape[2]):
        channel_data = X[:, :, channel]
        # Treat each row as a sample
        kpca = KernelPCA(n_components=n_components, kernel=kernel, gamma=gamma, fit_inverse_transform=True)
        transformed = kpca.fit_transform(channel_data)
        channel_reconstructed = kpca.inverse_transform(transformed)
        # Clip values to [0,1]
        channel_reconstructed = np.clip(channel_reconstructed, 0, 1)
        reconstructed_channels.append(channel_reconstructed)

    # Stack the reconstructed channels back into an image
    rec_img = np.stack(reconstructed_channels, axis=2)
    rec_img = (rec_img * 255).astype(np.uint8)
    return rec_img


# %% Kernel PCA loop on Kodak images
data_dir = Path("../data/Kodak")
image_paths = list(data_dir.glob("*.png"))
if not image_paths:
    raise FileNotFoundError("No images found in data/Kodak folder!")
for i in range(len(image_paths)):
    sample_image_path = image_paths[i]
    print("Using image:", sample_image_path)
    original_image = load_color_image(sample_image_path)

    # Apply Kernel PCA reconstruction
    n_components = 50  # adjust as needed for testing compression quality
    reconstructed_image = reconstruct_image_kernel_pca_color(original_image, n_components=n_components, kernel="rbf", gamma=0.01)

    # Compute evaluation metrics (PSNR and SSIM) on the luminance channel of the image
    def rgb2gray(rgb):
        return np.dot(rgb[..., :3], [0.2989, 0.5870, 0.1140])

    original_gray = rgb2gray(original_image)
    reconstructed_gray = rgb2gray(reconstructed_image)
    psnr_value = psnr(original_gray, reconstructed_gray, data_range=255)
    ssim_value = ssim(original_gray, reconstructed_gray, data_range=255)
    mse_value = mean_squared_error(original_gray, reconstructed_gray)
    print(f"Kernel PCA Reconstruction with {n_components} components per channel:")
    print("PSNR:", psnr_value)
    print("SSIM:", ssim_value)
    print("MSE:", mse_value)

    # Visualize the original and reconstructed images
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(original_image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    axes[1].imshow(reconstructed_image)
    axes[1].set_title(f"Reconstructed (Kernel PCA, n={n_components} per channel)")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()
    print("********************************************************")